In [24]:
from scipy.io import loadmat
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import models
from torch.utils.data import Dataset, DataLoader

In [3]:
def load_data(dataset):
    return loadmat(dataset)

In [4]:
# Load the dataset
training_data = load_data("Dataset/SVHN/train_32x32.mat")
test_data = load_data("Dataset/SVHN/test_32x32.mat")

In [5]:
x_train, y_train, x_test, y_test = training_data['X'], training_data['y'], test_data['X'], test_data['y']
x_train.shape, x_test.shape

((32, 32, 3, 73257), (32, 32, 3, 26032))

In [6]:
x_train, y_train, x_test, y_test = training_data['X'], training_data['y'], test_data['X'], test_data['y']

x_train = x_train.transpose((3, 2, 0, 1))
x_test = x_test.transpose((3, 2, 0, 1))

y_train = y_train[ : , 0]
y_test = y_test[ : , 0]

In [7]:
print(f"X_train: {x_train.shape}, y_train: {y_train.shape}, X_test: {x_test.shape}, y_test: {y_test.shape}")

X_train: (73257, 3, 32, 32), y_train: (73257,), X_test: (26032, 3, 32, 32), y_test: (26032,)


In [8]:
# change label 10 -> 0
def change_10_0(data):
    data[data == 10] = 0

In [9]:
change_10_0(y_train)
change_10_0(y_test)

In [10]:
x_train.shape

(73257, 3, 32, 32)

In [11]:
x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [12]:
x_train.shape

torch.Size([73257, 3, 32, 32])

In [13]:
# custom dataset
class Dataset(Dataset):
    
    def __init__(self, features, labels, transform = None):
        self.features = features.type(torch.FloatTensor)
        self.labels = labels.type(torch.LongTensor)
        
        # as the tensor we got from numpy is all int
        # self.features = self.features.type(torch.FloatTensor)
        # self.labels = self.labels.type(torch.FloatTensor)
        
        self.transform = transform
        
    def __len__(self):
        return (len(self.labels))
    
    def __getitem__(self, index):
        
        sample = self.features[index], self.labels[index]

        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [14]:
class transform(object):
    def __init__(self, divide_by = 255):
        self.divide_by = divide_by
        
    def __call__(self, sample):
        x = sample[0]
        y = sample[1]
        x = x / self.divide_by
        sample = x, y
        return sample

In [15]:
# train_loader and test_loader

transform = transform()

train_data = Dataset(x_train, y_train, transform = transform)
train_loader = DataLoader(train_data, batch_size = 64, shuffle = True, num_workers = 10)

test_data = Dataset(x_test, y_test, transform = transform)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = True, num_workers = 10)

In [16]:
"""class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 36, 3, padding = 1)
        self.pool = nn.MaxPool2d(2)
        self.dropout1 = nn.Dropout(p = 0.2)
        self.conv2 = nn.Conv2d(36, 64, 3, padding = 1)
        self.dropout2 = nn.Dropout(p = 0.4)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        
        self.fc1 = nn.Linear(128 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = F.relu(self.dropout1(self.pool(self.conv1(x))))
        x = F.relu(self.dropout2(self.pool(self.conv2(x))))
        
        x = F.relu(self.dropout2(self.pool(self.conv3(x))))
        
        #x = F.relu(self.dropout1(self.pool(self.conv4(x))))
        
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.log_softmax(self.fc5(x), dim = 1)
        
        
        return x"""

In [ ]:
"""model = CNN()
criterion = nn.NLLLoss()
images, labels = next(iter(train_loader))

print(images.shape, labels.shape)

logits = model(images)
print(logits.shape)
loss = criterion(logits, labels)"""

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [22]:

model = CNN()

model = model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.005)
epochs = 5
steps = 0
training_loss = 0
print_every = 2


for epoch in range(epochs):
    
    for images, labels in train_loader:
        steps += 1
        
        
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        log_ps = model(images)

        
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    
                    log_ps = model.forward(images)
                    batch_loss = criterion(log_ps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim = 1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
         f"Train loss: {training_loss / print_every:.3f}.. "
         f"Test loss: {test_loss / len(test_loader):.3f}.. "
         f"Test accuracy: {accuracy / len(test_loader):.3f}")
            
    training_loss = 0
    model.train()

Epoch 1/5.. Train loss: 2.290.. Test loss: 2.299.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 4.605.. Test loss: 2.298.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 6.912.. Test loss: 2.298.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 9.209.. Test loss: 2.298.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 11.514.. Test loss: 2.298.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 13.812.. Test loss: 2.298.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 16.108.. Test loss: 2.297.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 18.390.. Test loss: 2.297.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 20.700.. Test loss: 2.297.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 23.002.. Test loss: 2.297.. Test accuracy: 0.159
Epoch 1/5.. Train loss: 25.305.. Test loss: 2.297.. Test accuracy: 0.159


KeyboardInterrupt: 

In [27]:
model = models.densenet201(pretrained = True)

# freeze parameters so that we dont backprop through them
for param in model.parameters():
  param.requires_grad = False


model.classifier = nn.Sequential(nn.Linear(1920, 512),
                                nn.ReLU(),
                                nn.Dropout(0.2),
                                nn.Linear(512, 256),
                                nn.ReLU(),
                                nn.Dropout(0.2),
                                nn.Linear(256, 10),
                                nn.LogSoftmax(dim = 1))

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr = 0.005)

model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [28]:
epochs = 5
steps = 0
training_loss = 0
print_every = 2


for epoch in range(epochs):
    
    for images, labels in train_loader:
        steps += 1
        
        
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        log_ps = model(images)

        
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    
                    log_ps = model.forward(images)
                    batch_loss = criterion(log_ps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim = 1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
         f"Train loss: {training_loss / print_every:.3f}.. "
         f"Test loss: {test_loss / len(test_loader):.3f}.. "
         f"Test accuracy: {accuracy / len(test_loader):.3f}")
            
    training_loss = 0
    model.train()

Epoch 1/5.. Train loss: 2.792.. Test loss: 2.344.. Test accuracy: 0.112
Epoch 1/5.. Train loss: 5.111.. Test loss: 2.333.. Test accuracy: 0.111
Epoch 1/5.. Train loss: 7.377.. Test loss: 2.284.. Test accuracy: 0.111
Epoch 1/5.. Train loss: 9.666.. Test loss: 2.248.. Test accuracy: 0.199
Epoch 1/5.. Train loss: 11.913.. Test loss: 2.212.. Test accuracy: 0.197
Epoch 1/5.. Train loss: 14.101.. Test loss: 2.162.. Test accuracy: 0.278
Epoch 1/5.. Train loss: 16.270.. Test loss: 2.133.. Test accuracy: 0.267
Epoch 1/5.. Train loss: 18.348.. Test loss: 2.075.. Test accuracy: 0.281
Epoch 1/5.. Train loss: 20.457.. Test loss: 2.024.. Test accuracy: 0.283
Epoch 1/5.. Train loss: 22.531.. Test loss: 1.989.. Test accuracy: 0.306
Epoch 1/5.. Train loss: 24.517.. Test loss: 1.953.. Test accuracy: 0.306
Epoch 1/5.. Train loss: 26.432.. Test loss: 1.936.. Test accuracy: 0.295
Epoch 1/5.. Train loss: 28.219.. Test loss: 1.925.. Test accuracy: 0.321
Epoch 1/5.. Train loss: 30.065.. Test loss: 1.957.. Tes

Epoch 1/5.. Train loss: 199.802.. Test loss: 1.545.. Test accuracy: 0.453
Epoch 1/5.. Train loss: 201.364.. Test loss: 1.553.. Test accuracy: 0.446
Epoch 1/5.. Train loss: 202.811.. Test loss: 1.583.. Test accuracy: 0.444
Epoch 1/5.. Train loss: 204.428.. Test loss: 1.577.. Test accuracy: 0.448
Epoch 1/5.. Train loss: 206.146.. Test loss: 1.520.. Test accuracy: 0.471
Epoch 1/5.. Train loss: 207.640.. Test loss: 1.510.. Test accuracy: 0.472
Epoch 1/5.. Train loss: 209.239.. Test loss: 1.577.. Test accuracy: 0.449
Epoch 1/5.. Train loss: 210.879.. Test loss: 1.562.. Test accuracy: 0.454
Epoch 1/5.. Train loss: 212.553.. Test loss: 1.524.. Test accuracy: 0.472
Epoch 1/5.. Train loss: 214.160.. Test loss: 1.533.. Test accuracy: 0.465
Epoch 1/5.. Train loss: 215.785.. Test loss: 1.522.. Test accuracy: 0.460
Epoch 1/5.. Train loss: 217.237.. Test loss: 1.531.. Test accuracy: 0.450
Epoch 1/5.. Train loss: 218.876.. Test loss: 1.528.. Test accuracy: 0.451
Epoch 1/5.. Train loss: 220.478.. Test

Epoch 1/5.. Train loss: 372.501.. Test loss: 1.458.. Test accuracy: 0.481
Epoch 1/5.. Train loss: 374.100.. Test loss: 1.463.. Test accuracy: 0.474
Epoch 1/5.. Train loss: 375.621.. Test loss: 1.470.. Test accuracy: 0.474
Epoch 1/5.. Train loss: 377.174.. Test loss: 1.508.. Test accuracy: 0.469
Epoch 1/5.. Train loss: 378.690.. Test loss: 1.477.. Test accuracy: 0.477
Epoch 1/5.. Train loss: 380.283.. Test loss: 1.438.. Test accuracy: 0.489
Epoch 1/5.. Train loss: 381.783.. Test loss: 1.449.. Test accuracy: 0.486
Epoch 1/5.. Train loss: 383.111.. Test loss: 1.474.. Test accuracy: 0.478
Epoch 1/5.. Train loss: 384.703.. Test loss: 1.487.. Test accuracy: 0.472
Epoch 1/5.. Train loss: 386.275.. Test loss: 1.489.. Test accuracy: 0.478
Epoch 1/5.. Train loss: 387.728.. Test loss: 1.503.. Test accuracy: 0.483
Epoch 1/5.. Train loss: 389.037.. Test loss: 1.542.. Test accuracy: 0.471
Epoch 1/5.. Train loss: 390.672.. Test loss: 1.523.. Test accuracy: 0.479
Epoch 1/5.. Train loss: 392.144.. Test

KeyboardInterrupt: 